# PPM with implicit FCT
Apply FCT with implicit upwind to PPM with a large time step. 
The first application of FCT should use an implicit upwind method for the bounded solution. FCT then creates a bounded correction of PPM. This can be used as the bounded solution to apply FCT again to the PPM solution. Will this process converge to a more accurate bounded solution?

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import fluidfoam as foam
from initialConditions import *
from advectionSchemes import *
from fluxLimiters.fluxLimiters import *
matplotlib.rcParams['figure.dpi'] = 600
matplotlib.rcParams['figure.figsize'] =(8, 6)

In [ ]:
# Function for schemes comparisions

def compareSchemes(params, fluxes, options, labels, OFcases = [], OFtimes=[], OFfields=[],
                    OFlabels = []):
    """Solve the advection equation for various schemes and plot the results
    params: dict with entries "nt", "nx", "c", "initialConditions", "title", "fileName"
    fluxes: List of flux functions
    options: List of dictionaries to send to the flux functions
    labels:  Labels for the legend of the graph for each flux function
    """
    dt = params["c"]/params["nx"]
    dx = 1/params["nx"]
    print('Solving the advection equation for', dt*params["nt"],
          'revolutions of a periodic domain with spatial resolution', dx)
    x = np.arange(0,1, dx)
    phi0 = params["initialConditions"](x)
    phiE = params["initialConditions"]((x-dt*params["nt"])%1)
    #fig,ax = plt.subplots(1,2, figsize=(12,4), layout='constrained')
    #fig.sup
    plt.title(params["title"]+'\nc = '+str(round(params["c"],2))+' nx = '
              +str(params["nx"]) + ' nt = '+str(params["nt"]))
    plt.plot(x, phi0, 'k--', label = 't=0')
    plt.plot(x, phiE, 'k', label='t='+str(round(dt*params["nt"],2)))
    plt.axhline(y=0, color='k', ls=':', lw=0.5)
    plt.axhline(y=1, color='k', ls=':', lw=0.5)

    # Run all the schemes and plot the results
    lines = ['k-o', 'r-+', 'b-x', 'g--s', 'c--', 'm:', 'k:', 'r:', 'b:', 'c:', 'g:']
    for flux, name, option, line in zip(fluxes, labels, options, lines):
        phi = phi0.copy()
        for it in range(params["nt"]):
            phi = advect(phi, params["c"], flux, options=option)
        plt.plot(x, phi, line, label=name)
        #ax[1].plot(x, phi - phiE, label=name)

    # Add the OpenFOAM results
    lines = lines[len(fluxes):-1]
    for case, time, field, label, line in zip(OFcases, OFtimes,OFfields,OFlabels,lines):
        phi = foam.readscalar(case, str(time), field)
        plt.plot(x, phi, line, label=label)
    
    plt.legend()#bbox_to_anchor=(1.1, 1))
    #ax[0].set(ylabel=r'$\psi$', title = 'Totals', xlim=[0,1])
    #ax[1].set(ylabel='Error', title = 'Errors', xlim=[0,1])
    plt.xlim([0,1])
    plt.savefig(params["fileName"])
    plt.show()

# calculate the number of time steps from the number of revolutions, nx and c
nt = lambda nRevs, nx, c : int(nRevs*nx/c)


In [ ]:
# Explicit PPM with FCT and  MULES
compareSchemes(
    {"nt": nt(1,40,.4), "nx":40, "c":0.4, "initialConditions":combi,
     "title": "Explicit PPM Advection with FCT/MULES",
     "fileName": "plots/PPM_c04_FCT_MULES.pdf"},
    [FCT, FCT, MULES, MULES, PPMflux],
    [{"nCorr": 1}, {"nCorr": 2},
     {"nCorr": 1}, {"nCorr": 2}, {}],
    ['with 1 FCT', 'with 2 FCT', 'with 1 MULES1', 'with 2 MULES1', 'PPM'])


In [ ]:
# Explicit Centred differences with FCT and  MULES (not explicit)
compareSchemes(
    {"nt": nt(1,40,.4), "nx":40, "c":0.4, "initialConditions":combi,
     "title": r'Centred differences with FCT/MULES with $\alpha=\frac{1}{2}$ for CD',
     "fileName": "plots/CD_c04_FCT_MULES.pdf"},
    [FCT, FCT, MULES, MULES, CDFlux],
    [{'HO': CDFlux, "nCorr": 1, "explicit": False, "alpha": 0.5},
     {'HO': CDFlux, "nCorr": 2, "explicit": False, "alpha": 0.5},
     {'HO': CDFlux, "nCorr": 1, "explicit": False, "alpha": 0.5},
     {'HO': CDFlux, "nCorr": 2, "explicit": False, "alpha": 0.5},
     {"explicit": False, "alpha": 0.5}],
    ['CD with 1 FCT', 'CD with 2 FCT', 'CD with 1 MULES', 'CD with 2 MULES', 
     r'CD with $\alpha=\frac{1}{2}$'])

In [ ]:
# Explicit quasi-cubic with FCT and MULES (not explicit)
compareSchemes(
    {"nt" : nt(1,40,.4), "nx": 40, "c": 0.4, "initialConditions": combi,
     "title": r'Quasi-cubic with FCT/MULES with $\alpha=\frac{1}{2}$ for qC',
     "fileName": "plots/qC_c04_FCT_MULES.pdf"},
    [FCT, FCT, FCT, FCT, FCT, FCT, FCT, FCT, MULES, MULES, quasiCubicFlux],
    [{"nCorr": 1, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 2, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 3, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 4, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 5, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 6, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 7, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 8, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 1, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 2, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"explicit": False, "alpha": 0.5}],
    ['with 1 FCT', 'with 2 FCT', 'with 3 FCT', 'with 4 FCT', 'with 5 FCT', 'with 6 FCT', 
     'with 7 FCT', 'with 8 FCT', 
     'with 1 MULES', 'with 2 MULES', r'cubic, $\alpha=0.5$'])

In [ ]:
# Explicit linear upwind with FCT and MULES (not explicit)
compareSchemes(
    {"nt" : nt(1,40,.4), "nx": 40, "c": 0.4, "initialConditions": combi,
     "title": r'Linear-upwind with FCT/MULES with $\alpha=\frac{1}{2}$ for qC',
     "fileName": "plots/lu_c04_FCT_MULES.pdf"},
    [FCT, FCT, MULES, MULES, linearUpwindFlux],
    [{"nCorr": 1, "HO": linearUpwindFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 2, "HO": linearUpwindFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 1, "HO": linearUpwindFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 2, "HO": linearUpwindFlux, "explicit": False, "alpha": 0.5},
     {"explicit": False, "alpha": 0.5}],
    ['with 1 FCT', 'with 2 FCT', 'with 1 MULES', 
     'with 2 MULES', r'linear uwpind, $\alpha=0.5$'])

In [ ]:
# PPM with iterations of implicit, monotonic FCT with c = 1.4
compareSchemes(
    {"nt": 28, "nx":40, "c":40/28, "initialConditions":combi,
     "title": "Implicit Advection with FCT",
     "fileName": "plots/PPM_c14_FCT.pdf"},
    [PPMflux, upwindFlux, FCT, FCT, FCT],
    [{}, {}, {"nCorr": 1}, {"nCorr": 2}, {"nCorr": 3}],
    ['PPM', 'upwind', 'PPM with 1 FCT', 'PPM with 2 FCT', 'PPM with 3 FCT'])

In [ ]:
# PPM with iterations of implicit, monotonic MULES with c = 1.4
compareSchemes(
    {"nt": 28, "nx":40, "c":40/28, "initialConditions":combi,
     "title": "Implicit Advection with MULES",
     "fileName": "plots/PPM_c14_MULES.pdf"},
    [PPMflux, upwindFlux, MULES, MULES, MULES],
    [{}, {}, {"nCorr": 1}, {"nCorr": 2}, {"nCorr": 3,  "MULES": True}],
    ['PPM', 'upwind', 'PPM with 1 MULES', 'PPM with 2 MULES', 'PPM with 3 MULES'])

In [ ]:
# PPM with iterations of implicit, monotonic FCT with c = 2.4
compareSchemes(
    {"nt": 32, "nx":80, "c":80/32, "initialConditions":combi,
     "title": "Implicit Advection with FCT",
     "fileName": "plots/PPM_c24_FCT.pdf"},
    [PPMflux, upwindFlux, FCT, FCT, FCT, FCT, FCT, FCT],
    [{}, {}, {"nCorr": 1}, {"nCorr": 2}, {"nCorr": 3}, {"nCorr": 4}, {"nCorr": 5},
     {"nCorr": 6}],
    ['PPM', 'upwind', 'PPM with 1 FCT', 'PPM with 2 FCT', 'PPM with 3 FCT',
     'PPM with 4 FCT', 'PPM with 5 FCT', 'PPM with 6 FCT'])

In [ ]:
# PPM with iterations of implicit, monotonic MULES with c = 2.4
compareSchemes(
    {"nt": 32, "nx":80, "c":80/32, "initialConditions":combi,
     "title": "Implicit Advection with MULES",
     "fileName": "plots/PPM_c24_MULES.pdf"},
    [PPMflux, upwindFlux, MULES, MULES, MULES, MULES, MULES, MULES],
    [{}, {}, {"nCorr": 1}, {"nCorr": 2}, {"nCorr": 3}, {"nCorr": 4},  {"nCorr": 5},{"nCorr": 6}],
    ['PPM', 'upwind', 'PPM with 1 MULES', 'PPM with 2 MULES', 'PPM with 3 MULES',
     'PPM with 4 MULES', 'PPM with 5 MULES', 'PPM with 6 MULES'])

In [ ]:
# PPM with iterations of 2 versions of MULES
compareSchemes(
    {"nt": 1, "nx":40, "c":40/16, "initialConditions":combi,
     "title": "Advection with MULES",
     "fileName": "plots/PPM_c24_MULES_2.pdf"},
    [MULES, MULES, MULES, MULES],
    [{"nCorr": 1}, {"nCorr": 2}, 
     {"nCorr": 1, "safeStart": True},  {"nCorr": 2, "safeStart": True}],
    ['PPM with 1 MULES', 'PPM with 2 MULES', 
     'PPM with 1 MULES2', 'PPM with 2 MULES2'])

In [ ]:
# CD with iterations of 2 versions of MULES and FCT
from fluxLimiters.fluxLimiters import *
compareSchemes(
    {"nt": 1, "nx":40, "c":40/16, "initialConditions":halfWave,
     "title": "Advection with Central Differencing",
     "fileName": "plots/CD_c25_MULES_2.pdf"},
    [MULES, MULES, FCT, FCT, FCT, FCT],
    [{"HO": CDFlux, "nCorr": 2}, 
     {"HO": CDFlux, "nCorr": 2, "safeStart": True},
     {"HO": CDFlux, "nCorr": 1}, {"HO": CDFlux, "nCorr": 2},
     {"HO": CDFlux, "nCorr": 3}, {"HO": CDFlux, "nCorr": 4}],
    ['CD with 2 MULES', 'CD with 2 MULES2', 
     'CD with 1 FCT', 'CD with 2 FCT', 'CD with 3 FCT', 'CD with 4 FCT'])

In [ ]:
from advectionSchemes import *
from fluxLimiters.fluxLimiters import *
compareSchemes(
    {"nt": 1, "nx":40, "c":40/16, "initialConditions":halfWave,
     "title": "Advection with Central Differencing",
     "fileName": "plots/CD_c25.pdf"},
    [FCT, FCT, MULES, MULES],
    [{"HO": CDFlux, "nCorr": 1}, {"HO": CDFlux, "nCorr": 2}, 
     {"HO": CDFlux, "nCorr": 2}, {"HO": CDFlux, "nCorr": 2, "safeStart": True}],
    ['CD with 1 FCT', 'CD with 2 FCT',
     'CD with 2 MULES', 'CD with 2 MULES2'],
    OFcases = ['/home/hilary/OpenFOAM/hilary-dev/run/advection/halfWave'],
    OFtimes=[0.0625], OFfields=['T'], OFlabels = ['OpenFOAM MULES'])

In [ ]:
# Comparison with Amber's results on doubleFCT.py
compareSchemes(
    {"nt" : int(100/6.25), "nx": 40, "c": 2.5, "initialConditions": combi,
     "title": "Quasi-cubic advection with Trapezoidal-implicit",
     "fileName": "plots/qC_c2p5_FCT.pdf"},
    [quasiCubicFlux, FCT, FCT, FCT, FCT, FCT, FCT],
    [{"explicit": False, "alpha": 0.5},
     {"nCorr": 1, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 2, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 3, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 4, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 5, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5},
     {"nCorr": 6, "HO": quasiCubicFlux, "explicit": False, "alpha": 0.5}],
    [r'cubic, $\alpha=0.5$', 'with 1 FCT', 'with 2 FCT', 'with 3 FCT', 
     'with 4 FCT', 'with 5 FCT', 'with 6 FCT'])